<a href="https://colab.research.google.com/github/josh-kindhart/Python-Practice/blob/main/First_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('medical_costs.csv')
print(df)

      age  sex     bmi  children  smoker     region      charges
0      19    1  27.900         0       1  southwest  16884.92400
1      18    0  33.770         1       0  southeast   1725.55230
2      28    0  33.000         3       0  southeast   4449.46200
3      33    0  22.705         0       0   0rthwest  21984.47061
4      32    0  28.880         0       0   0rthwest   3866.85520
...   ...  ...     ...       ...     ...        ...          ...
1333   50    0  30.970         3       0   0rthwest  10600.54830
1334   18    1  31.920         0       0   0rtheast   2205.98080
1335   18    1  36.850         0       0  southeast   1629.83350
1336   21    1  25.800         0       0  southwest   2007.94500
1337   61    1  29.070         0       1   0rthwest  29141.36030

[1338 rows x 7 columns]


In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 73.3+ KB
None


In [4]:
print(df.region.unique())

['southwest' 'southeast' '0rthwest' '0rtheast']


In [5]:
df.replace('0rthwest', 'northwest', inplace=True)
df.replace('0rtheast','northeast', inplace=True)

df = df.rename(columns={'sex':'gender'})
df.gender.replace(1, 'female', inplace=True)
df.gender.replace(0, 'male', inplace=True)

df.gender = pd.Categorical(df.gender, ['male','female'], ordered=False)
df.region = pd.Categorical(df.region, list(df.region.unique()), ordered=False)

In [6]:
features = df.iloc[:, 0:6]
labels = df.iloc[:, -1]

In [7]:
features = pd.get_dummies(features)

In [8]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [9]:
ct = ColumnTransformer([('normalize', Normalizer(), ['age','bmi','children'])], remainder='passthrough')
features_train = ct.fit_transform(features_train)
features_test = ct.transform(features_test)

In [10]:
def design_model(features_train):
  model = Sequential(name="my_first_neural_network")
  num_features = features_train.shape[1]
  input = InputLayer(input_shape=(num_features,))
  model.add(input)
  model.add(Dense(128, activation='relu'))
  model.add(Dense(1))
  return model

In [11]:
model = design_model(features_train)
opt = Adam(learning_rate=0.01)
model.compile(loss='mse', metrics=['mae'], optimizer=opt)
model.fit(features_train, labels_train, batch_size=1, epochs=100, verbose=1)

Epoch 1/100
896/896 [==============================] - 3s 3ms/step - loss: 217135536.0000 - mae: 9945.4346
Epoch 2/100
896/896 [==============================] - 2s 2ms/step - loss: 122460104.0000 - mae: 8159.0356
Epoch 3/100
896/896 [==============================] - 2s 2ms/step - loss: 109200440.0000 - mae: 7872.1992
Epoch 4/100
896/896 [==============================] - 3s 3ms/step - loss: 97468496.0000 - mae: 7388.6177
Epoch 5/100
896/896 [==============================] - 1s 2ms/step - loss: 86258728.0000 - mae: 7098.4736
Epoch 6/100
896/896 [==============================] - 1s 1ms/step - loss: 76177456.0000 - mae: 6613.2583
Epoch 7/100
896/896 [==============================] - 1s 1ms/step - loss: 67643336.0000 - mae: 6353.1753
Epoch 8/100
896/896 [==============================] - 1s 1ms/step - loss: 61466584.0000 - mae: 5912.5361
Epoch 9/100
896/896 [==============================] - 1s 1ms/step - loss: 57845204.0000 - mae: 5854.0625
Epoch 10/100
896/896 [=====================

In [12]:
val_mse, val_mae = model.evaluate(features_test, labels_test, verbose=1)

14/14 [==============================] - 0s 1ms/step - loss: 48277648.0000 - mae: 4742.0254


In [13]:
print(f'MSE: {val_mse}, MAE: {val_mae}')

MSE: 48277648.0, MAE: 4742.025390625


In [14]:
print(model.summary())

Model: "my_first_neural_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________
None
